In [1]:
##################
#####################
import pysam, os
import pandas as pd
from glob import glob
from tqdm import tqdm
from datetime import datetime
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
# load all sites extracted from kidney
columns = ["Region", "Position", "Ref", "Strand", "Cov", "Qual", "[A,C,G,T]", "AllSubs", "Freq", "gCov", "gQual", "g[A,C,G,T]", "gAllSubs", "gFreq", "type", "sample"]
print(len(columns))
df_sites_all = pd.read_table("kidney_extracted_pos_neg_sites_rediportal_13112023.tsv", index_col=0, names=columns, skiprows=1)
df_sites_all

16


,Region,Position,Ref,Strand,Cov,Qual,"[A,C,G,T]",AllSubs,Freq,gCov,gQual,"g[A,C,G,T]",gAllSubs,gFreq,type,sample
0,chr1,887801,A,2,58,38.74,"[0, 0, 58, 0]",AG,1.00,42,28.88,"[0, 0, 42, 0]",AG,1.0,SNP,SRR1071807
1,chr1,981931,A,2,56,38.34,"[0, 0, 56, 0]",AG,1.00,23,28.61,"[0, 0, 23, 0]",AG,1.0,SNP,SRR1071807
2,chr1,1271175,A,2,135,37.31,"[0, 0, 135, 0]",AG,1.00,28,27.36,"[0, 0, 28, 0]",AG,1.0,SNP,SRR1071807
3,chr1,1288345,A,2,246,38.83,"[0, 0, 246, 0]",AG,1.00,47,29.02,"[0, 0, 47, 0]",AG,1.0,SNP,SRR1071807
4,chr1,1288823,A,2,136,37.49,"[0, 0, 136, 0]",AG,1.00,34,28.09,"[0, 0, 34, 0]",AG,1.0,SNP,SRR1071807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306746,chrY,21869330,A,2,72,37.06,"[70, 0, 2, 0]",AG,0.03,21,35.76,"[21, 0, 0, 0]",-,0.0,Editing,SRR821356
1306747,chrY,21869415,A,2,76,37.67,"[73, 0, 3, 0]",AG,0.04,15,36.87,"[15, 0, 0, 0]",-,0.0,Editing,SRR821356
1306748,chrY,21869628,A,2,59,36.02,"[58, 0, 1, 0]",AG,0.02,16,36.75,"[16, 0, 0, 0]",-,0.0,Editing,SRR821356
1306749,chrY,21870242,A,2,72,36.71,"[70, 0, 2, 0]",AG,0.03,21,38.00,"[21, 0, 0, 0]",-,0.0,Editing,SRR821356


In [3]:
# detect number of samples
df_sites_all["sample"].value_counts().shape

(31,)

In [4]:
# create ohe instance
ohe = OneHotEncoder()
ohe.fit(np.array(["A", "C", "G", "T"]).reshape(-1, 1))

features_extracted_filepath = "kidney_extracted_pos_neg_sites_rediportal_13112023.feature_vectors.tsv"
features_extracted = open(features_extracted_filepath, "w")
intervals = []
total_snps = 0
total_editing = 0
interval = 101
start_time_global = datetime.now()
for n,sample in enumerate(df_sites_all["sample"].value_counts().index):
    print(f"\n########################\n[{datetime.now()}] Processing sample:", sample)
    df = df_sites_all.query(f"sample == '{sample}'")
    # mantain only non chrM regions
    df = df.query("Region != 'chrM'")
    srr_filepath = glob( os.path.join('/lustre/biomed/epicardi/ncbi/dbGaP-6698/sra',sample,sample+'_dna.txt.gz'))[0]
    print(f"[{datetime.now()}] Loading reditable with tabix and pysam:", srr_filepath)
    start_time = datetime.now()
    srr = pysam.TabixFile(srr_filepath)
    # extract pos examples
    with tqdm(total=df.shape[0], position=0, leave=True) as pbar:
        for site in df.query("type == 'Editing'").itertuples():
            start = site.Position - ((interval-1)/2)
            stop = site.Position + ((interval-1)/2)
            AGrna = eval(site._7)[2]/sum(eval(site._7))
            AGwgs = eval(site._12)[2]/sum(eval(site._12))
            srr_interval = []
            for s in srr.fetch(site.Region, start-1, stop):
                srr_interval.append(s.split("\t"))
            srr_interval = pd.DataFrame(srr_interval, columns=columns[:-2])
            # assess wheter interval is of the required length and contains at least 2 AG subs
            if srr_interval.shape[0] == interval and srr_interval.AllSubs.value_counts().reset_index()[srr_interval.AllSubs.value_counts().reset_index()["index"].str.contains("AG")].shape[0] >= 2:
                intervals.append([site.Region, site.Position, site.Strand, AGrna, AGwgs, site.type, site.sample, start, stop, stop-start + 1, srr_interval.shape[0]])
                total_editing += 1
                # encode features vector and write to disk
                seq = srr_interval.Ref.values.reshape(-1,1)
                seq_ohe = ohe.transform(seq).toarray().T
                vects_freqs = []
                vects = []
                for vect in srr_interval["[A,C,G,T]"]:
                    vect = np.array(eval(vect))
                    cov = sum(vect)
                    vect_freqs = vect / cov
                    vects_freqs.append(vect_freqs)
                    vects.append(vect)
                vects_freqs = np.array(vects_freqs).T
                vects = np.array(vects).T
                site = pd.concat([pd.DataFrame(seq_ohe), pd.DataFrame(vects_freqs)])
                # save to disk (append mode)
                site.to_csv(features_extracted, mode="a", sep="\t", header = None, index=None)
            pbar.update(1)
        # extract neg examples
        for site in df.query("type == 'SNP'").itertuples():
            start = site.Position - ((interval-1)/2)
            stop = site.Position + ((interval-1)/2)
            AGrna = eval(site._7)[2]/sum(eval(site._7))
            AGwgs = eval(site._12)[2]/sum(eval(site._12))
            srr_interval = []
            for s in srr.fetch(site.Region, start-1, stop):
                srr_interval.append(s.split("\t"))
            srr_interval = pd.DataFrame(srr_interval, columns=columns[:-2])
            # assess wheter interval is of the required length
            if srr_interval.shape[0] == interval:
                intervals.append([site.Region, site.Position, site.Strand, AGrna, AGwgs, site.type, site.sample, start, stop, stop-start + 1, srr_interval.shape[0]])
                total_snps += 1
                # encode features vector and write to disk
                seq = srr_interval.Ref.values.reshape(-1,1)
                seq_ohe = ohe.transform(seq).toarray().T
                vects_freqs = []
                vects = []
                for vect in srr_interval["[A,C,G,T]"]:
                    vect = np.array(eval(vect))
                    cov = sum(vect)
                    vect_freqs = vect / cov
                    vects_freqs.append(vect_freqs)
                    vects.append(vect)
                vects_freqs = np.array(vects_freqs).T
                vects = np.array(vects).T
                site = pd.concat([pd.DataFrame(seq_ohe), pd.DataFrame(vects_freqs)])
                # save to disk (append mode)
                site.to_csv(features_extracted, mode="a", sep="\t", header = None, index=None)
            pbar.update(1)
        print(f"[{datetime.now()}] Computation for sample {sample} finished. Elapsed time: {datetime.now()-start_time}")
        print(f"[{datetime.now()}] Total extracted Editing sites: {total_editing}. Total extracted SNPs: {total_snps}. Samples processed: {n+1}")
stop_time_global = datetime.now()
print(f"[{datetime.now()}] Computation Finished. Elapsed time {stop_time_global-start_time_global}")
features_extracted.close()


########################
[2023-11-14 02:56:52.008185] Processing sample: SRR1432650
[2023-11-14 02:56:52.046440] Loading reditable with tabix and pysam: /lustre/biomed/epicardi/ncbi/dbGaP-6698/sra/SRR1432650/SRR1432650_dna.txt.gz


100%|██████████| 64134/64134 [03:23<00:00, 314.88it/s]


[2023-11-14 03:00:15.749154] Computation for sample SRR1432650 finished. Elapsed time: 0:03:23.702628
[2023-11-14 03:00:15.749259] Total extracted Editing sites: 3654. Total extracted SNPs: 737. Samples processed: 1

########################
[2023-11-14 03:00:15.749795] Processing sample: SRR1317086
[2023-11-14 03:00:15.791549] Loading reditable with tabix and pysam: /lustre/biomed/epicardi/ncbi/dbGaP-6698/sra/SRR1317086/SRR1317086_dna.txt.gz


100%|██████████| 61595/61595 [03:16<00:00, 313.32it/s]


[2023-11-14 03:03:32.405986] Computation for sample SRR1317086 finished. Elapsed time: 0:03:16.614359
[2023-11-14 03:03:32.406025] Total extracted Editing sites: 5909. Total extracted SNPs: 1288. Samples processed: 2

########################
[2023-11-14 03:03:32.407223] Processing sample: SRR1089504
[2023-11-14 03:03:32.451430] Loading reditable with tabix and pysam: /lustre/biomed/epicardi/ncbi/dbGaP-6698/sra/SRR1089504/SRR1089504_dna.txt.gz


...

[2023-11-14 04:04:17.917208] Computation for sample SRR809943 finished. Elapsed time: 0:00:51.585864
[2023-11-14 04:04:17.917252] Total extracted Editing sites: 43225. Total extracted SNPs: 14052. Samples processed: 29

########################
[2023-11-14 04:04:17.917863] Processing sample: SRR1442708
[2023-11-14 04:04:17.951060] Loading reditable with tabix and pysam: /lustre/biomed/epicardi/ncbi/dbGaP-6698/sra/SRR1442708/SRR1442708_dna.txt.gz


100%|██████████| 20900/20900 [01:01<00:00, 338.92it/s]


[2023-11-14 04:05:19.641977] Computation for sample SRR1442708 finished. Elapsed time: 0:01:01.690868
[2023-11-14 04:05:19.642019] Total extracted Editing sites: 43741. Total extracted SNPs: 14244. Samples processed: 30

########################
[2023-11-14 04:05:19.642718] Processing sample: SRR1437274
[2023-11-14 04:05:19.673383] Loading reditable with tabix and pysam: /lustre/biomed/epicardi/ncbi/dbGaP-6698/sra/SRR1437274/SRR1437274_dna.txt.gz


100%|██████████| 7566/7566 [00:17<00:00, 442.96it/s]

[2023-11-14 04:05:36.776516] Computation for sample SRR1437274 finished. Elapsed time: 0:00:17.103101
[2023-11-14 04:05:36.776552] Total extracted Editing sites: 43898. Total extracted SNPs: 14321. Samples processed: 31
[2023-11-14 04:05:36.777352] Computation Finished. Elapsed time 1:08:44.829018


In [5]:
intervals = pd.DataFrame(intervals, columns=["Region", "Position", "Strand", "AGrna", "AGwgs", "Type", "SRR", "Start", "Stop", "DeltaStartStop", "TabixIntervalLen"])
intervals

,Region,Position,Strand,AGrna,AGwgs,Type,SRR,Start,Stop,DeltaStartStop,TabixIntervalLen
0,chr1,880977,2,0.010676,0.000000,Editing,SRR1432650,880927.0,881027.0,101.0,101
1,chr1,934686,2,0.018868,0.000000,Editing,SRR1432650,934636.0,934736.0,101.0,101
2,chr1,934696,2,0.017241,0.000000,Editing,SRR1432650,934646.0,934746.0,101.0,101
3,chr1,934723,2,0.010870,0.000000,Editing,SRR1432650,934673.0,934773.0,101.0,101
4,chr1,991238,2,0.010319,0.000000,Editing,SRR1432650,991188.0,991288.0,101.0,101
...,...,...,...,...,...,...,...,...,...,...,...
58214,chr22,24936970,2,0.477273,0.482143,SNP,SRR1437274,24936920.0,24937020.0,101.0,101
58215,chr22,24938908,2,0.476923,0.500000,SNP,SRR1437274,24938858.0,24938958.0,101.0,101
58216,chrX,43604841,2,1.000000,1.000000,SNP,SRR1437274,43604791.0,43604891.0,101.0,101
58217,chrX,153629155,2,1.000000,1.000000,SNP,SRR1437274,153629105.0,153629205.0,101.0,101


In [6]:
!cat kidney_extracted_pos_neg_sites_rediportal_13112023.feature_vectors.tsv | wc -l

465752


In [7]:
# save intervals as response variable and metadata
intervals.to_csv("kidney_extracted_pos_neg_sites_rediportal_13112023.feature_vectors.metadata.tsv", sep="\t", index=None)

In [8]:
!cat kidney_extracted_pos_neg_sites_rediportal_13112023.feature_vectors.metadata.tsv | head

Region	Position	Strand	AGrna	AGwgs	Type	SRR	Start	Stop	DeltaStartStop	TabixIntervalLen
chr1	880977	2	0.010676156583629894	0.0	Editing	SRR1432650	880927.0	881027.0	101.0	101
chr1	934686	2	0.018867924528301886	0.0	Editing	SRR1432650	934636.0	934736.0	101.0	101
chr1	934696	2	0.017241379310344827	0.0	Editing	SRR1432650	934646.0	934746.0	101.0	101
chr1	934723	2	0.010869565217391304	0.0	Editing	SRR1432650	934673.0	934773.0	101.0	101
chr1	991238	2	0.010318949343339587	0.0	Editing	SRR1432650	991188.0	991288.0	101.0	101
chr1	1191495	2	0.0205761316872428	0.0	Editing	SRR1432650	1191445.0	1191545.0	101.0	101
chr1	1248165	2	0.024691358024691357	0.0	Editing	SRR1432650	1248115.0	1248215.0	101.0	101
chr1	1288523	2	0.01171875	0.0	Editing	SRR1432650	1288473.0	1288573.0	101.0	101
chr1	1309144	2	0.035211267605633804	0.0	Editing	SRR1432650	1309094.0	1309194.0	101.0	101
cat: write error: Broken pipe


In [9]:
!cat kidney_extracted_pos_neg_sites_rediportal_13112023.feature_vectors.metadata.tsv | wc -l

58220


In [10]:
intervals["Type"].value_counts()

Editing    43898
SNP        14321
Name: Type, dtype: int64

In [15]:
intervals.groupby("Type").describe()[["AGrna", "AGwgs"]].T

Type              Editing           SNP
AGrna count  43898.000000  14321.000000
      mean       0.026781      0.914030
      std        0.056509      0.187875
      min        0.010000      0.380000
      25%        0.011858      0.994318
      50%        0.014749      1.000000
      75%        0.020408      1.000000
      max        1.000000      1.000000
AGwgs count  43898.000000  14321.000000
      mean       0.000000      0.914845
      std        0.000000      0.187742
      min        0.000000      0.400000
      25%        0.000000      1.000000
      50%        0.000000      1.000000
      75%        0.000000      1.000000
      max        0.000000      1.000000

In [16]:
43898 + 14321

58219.0

In [ ]:
# load features
feat = pd.read_table()